<a href="https://colab.research.google.com/github/karjain/Anti_deepFake/blob/main/datapipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import argparse
from os.path import join
import cv2
import dlib
import glob
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive




---


The following function picks up frames from the video and detects face in it. If a face is found, it is cropped and stored in the REAL or FAKE folder according to the videos label.

The labels are dected from the metafile


---



In [ ]:
def video_to_croped_frames(video_files_root, out_path, metafile_path, frame_rate=33):
  file_extensions = ['mp4','gif','webm','avi','3gp','wmv','flv','mkv']
  #file_extension eg 'mp4', 'avi' 
  
  #Read labels from a meta files
  df = pd.read_json(metafile_path, orient='index')

  video_files_path=[]
  for ext in file_extensions:
    video_files_path.extend(glob.glob(video_files_root + '/*.'+ext))
    print(type(video_files_path))
  no_files = len(video_files_path)

  #os.makedirs(out_path+'/fake', exist_ok=True)
  #os.makedirs(out_path+'/real', exist_ok=True)
  
  videos_processed=0

  for video_file in video_files_path:
    reader = cv2.VideoCapture(video_file)
    
    video_name = video_file.split('/')[-1]
    video_fn = video_name.split('.')[0]

    label = df.loc[video_name,'label']

    #fps = reader.get(cv2.CAP_PROP_FPS)
    #num_frames = int(reader.get(cv2.CAP_PROP_FRAME_COUNT))
    #print("num of frames: ",num_frames)
  
    # Face detector
    face_detector = dlib.get_frontal_face_detector()
    
    # Frame numbers and length of output video
    i=0
    c=1
    while reader.isOpened():
        _, image = reader.read()
        if image is None:
            break
        if c%frame_rate==0:
    
          # Image size
          height, width = image.shape[:2]

          # . Detect with dlib
          gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
          faces = face_detector(gray, 1)
          if len(faces):
              for face in faces:
                # Face crop with dlib and bounding box scale enlargement
                x, y, size = get_boundingbox(face, width, height)
                cropped_face = image[y:y+size, x:x+size]
                cropped_face = cv2.resize(cropped_face, (112, 112))
                if( label=='FAKE' ):
                  cv2.imwrite(out_path + "/fake/" + "deepF" + video_fn+str(i) + '.jpg', cropped_face)
                else:
                  cv2.imwrite(out_path + "/real/" + "deepF" + video_fn+str(i) + '.jpg', cropped_face)
                
                i+=1
                #cv2.waitKey(33)
          
        c+=1
    print(video_name, " ", label)
    print("total frames written: ",i)
    videos_processed+=1
    print("Progress= ", (videos_processed/no_files)*100  )
      
  

In [ ]:
video_to_croped_frames('/content/drive/MyDrive/Fake_dect/deepF/train_vid', '/content/drive/MyDrive/Fake_dect/merged_data', '/content/drive/MyDrive/Fake_dect/deepF/train_vid/metadata.json', frame_rate=33)